In [1]:
# ECS289G_Term_Project/data_cleaning.ipynb

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
file_path = r"/Users/harshil/Developer/GitHub_Repos/ECS_289G/data/raw/state_CA-WA-NJ-CT.csv"
df = pd.read_csv(file_path, low_memory=False)

# Display initial dataset information
print(f"Dataset Shape: {df.shape}")
df.head()
df.info()

Dataset Shape: (1602362, 99)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1602362 entries, 0 to 1602361
Data columns (total 99 columns):
 #   Column                                    Non-Null Count    Dtype  
---  ------                                    --------------    -----  
 0   activity_year                             1602362 non-null  int64  
 1   lei                                       1602362 non-null  object 
 2   derived_msa-md                            1602362 non-null  int64  
 3   state_code                                1602362 non-null  object 
 4   county_code                               1589888 non-null  float64
 5   census_tract                              1586190 non-null  float64
 6   conforming_loan_limit                     1594356 non-null  object 
 7   derived_loan_product_type                 1602362 non-null  object 
 8   derived_dwelling_category                 1602362 non-null  object 
 9   derived_ethnicity                         1602362 

In [2]:
# --- Step 1: Selecting Relevant Columns ---
# Columns to retain
columns_to_keep = [
    'tract_to_msa_income_percentage', 'ffiec_msa_md_median_family_income',
    'tract_minority_population_percent', 'interest_rate',
    'loan_type', 'loan_purpose', 'lien_status',
    'construction_method', 'occupancy_type',
    'derived_ethnicity', 'derived_race', 'derived_sex',
    'action_taken'
]

# Filter the dataset
filtered_data = df[columns_to_keep].copy()

# Verify selected columns
print(f"Filtered Dataset Shape: {filtered_data.shape}")

Filtered Dataset Shape: (1602362, 13)


In [3]:
# --- Step 2: Handling Missing Values ---
# Fill missing demographic values with placeholders
demographic_features = ['derived_ethnicity', 'derived_race', 'derived_sex']
for col in demographic_features:
    filtered_data[col] = filtered_data[col].fillna(f"{col.split('_')[-1]} Not Available")

# Handle non-numeric values in 'interest_rate'
filtered_data['interest_rate'] = pd.to_numeric(filtered_data['interest_rate'], errors='coerce')

# Check missing values
print("Missing Values Before Handling:")
print(filtered_data.isnull().sum())

Missing Values Before Handling:
tract_to_msa_income_percentage            0
ffiec_msa_md_median_family_income         0
tract_minority_population_percent         0
interest_rate                        580347
loan_type                                 0
loan_purpose                              0
lien_status                               0
construction_method                       0
occupancy_type                            0
derived_ethnicity                         0
derived_race                              0
derived_sex                               0
action_taken                              0
dtype: int64


In [4]:
# Step 2.1: Impute Missing 'interest_rate' Values with Median

# Calculate the median of the 'interest_rate' column
interest_rate_median = filtered_data['interest_rate'].median()
print(f"Median Interest Rate: {interest_rate_median}")

# Fill missing 'interest_rate' values with the median (avoiding inplace assignment)
filtered_data['interest_rate'] = filtered_data['interest_rate'].fillna(interest_rate_median)

# Verify that there are no more missing values in 'interest_rate'
missing_interest_rate_after = filtered_data['interest_rate'].isnull().sum()
print(f"Missing 'interest_rate' values after imputation: {missing_interest_rate_after}")

Median Interest Rate: 6.75
Missing 'interest_rate' values after imputation: 0


In [5]:
# Check missing values
print("Missing Values Before Handling:")
print(filtered_data.isnull().sum())

Missing Values Before Handling:
tract_to_msa_income_percentage       0
ffiec_msa_md_median_family_income    0
tract_minority_population_percent    0
interest_rate                        0
loan_type                            0
loan_purpose                         0
lien_status                          0
construction_method                  0
occupancy_type                       0
derived_ethnicity                    0
derived_race                         0
derived_sex                          0
action_taken                         0
dtype: int64


In [6]:
# --- Step 3: Encoding Demographic Features ---
# Define encoding mappings
sex_mapping = {
    "Male": 0, "Female": 1, "Joint": 2, "Sex Not Available": 3
}

race_mapping = {
    "American Indian or Alaska Native": 0,
    "Asian": 1,
    "Black or African American": 2,
    "Native Hawaiian or Other Pacific Islander": 3,
    "White": 4,
    "2 or more minority races": 5,
    "Joint": 6,
    "Free Form Text Only": 7,
    "Race Not Available": 8
}

ethnicity_mapping = {
    "Hispanic or Latino": 0,
    "Not Hispanic or Latino": 1,
    "Joint": 2,
    "Ethnicity Not Available": 3,
    "Free Form Text Only": 4
}

# Map demographic features using the provided mappings
filtered_data["derived_sex"] = filtered_data["derived_sex"].map(sex_mapping)
filtered_data["derived_race"] = filtered_data["derived_race"].map(race_mapping)
filtered_data["derived_ethnicity"] = filtered_data["derived_ethnicity"].map(ethnicity_mapping)

# One-hot encoding for demographic features
race_one_hot = pd.get_dummies(filtered_data['derived_race'], prefix='race')
gender_one_hot = pd.get_dummies(filtered_data['derived_sex'], prefix='gender')
ethnicity_one_hot = pd.get_dummies(filtered_data['derived_ethnicity'], prefix='ethnicity')

# Combine with the main dataset
filtered_data = pd.concat([filtered_data, race_one_hot, gender_one_hot, ethnicity_one_hot], axis=1)

# Drop original demographic columns
filtered_data.drop(columns=['derived_race', 'derived_sex', 'derived_ethnicity'], inplace=True)

# --- Step 4: Encoding Other Categorical Features ---
# One-hot encoding for other categorical features
filtered_data = pd.get_dummies(filtered_data, columns=['loan_type', 'loan_purpose', 'lien_status',
                                                       'construction_method', 'occupancy_type'], drop_first=True)

# Verify transformations
print("Data After One-Hot Encoding:")
print(filtered_data.head())


Data After One-Hot Encoding:
   tract_to_msa_income_percentage  ffiec_msa_md_median_family_income  \
0                          131.97                              76600   
1                          128.98                             101700   
2                           62.21                              94500   
3                          108.93                             113900   
4                           69.90                              94500   

   tract_minority_population_percent  interest_rate  action_taken  race_0  \
0                              19.02          6.000             6   False   
1                              26.74          5.875             6   False   
2                              49.51          6.000             6   False   
3                              26.27          6.000             6   False   
4                              43.85          6.375             6   False   

   race_1  race_2  race_3  race_4  ...  loan_type_4  loan_purpose_2  \
0   

In [ ]:
# --- Step 5: Saving the Cleaned Dataset ---
output_path = r"/Users/harshil/Developer/GitHub_Repos/ECS_289G/data/processsed/cleaned/cleaned_dataset_yrs-23.csv"
filtered_data.to_csv(output_path, index=False)

print(f"Cleaned dataset saved to {output_path}")

Cleaned dataset saved to /Users/harshil/Documents/UC Davis/Fall Quarter '24/ECS 289G/term_project/root_folder_1/data/processsed/cleaned_dataset_yrs-23.csv
